In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from pandas import HDFStore

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    
path_to_hdf_datastore = cfg['path_to_hdf_datastore']
path_to_clean_hdf_datastore = cfg['path_to_clean_hdf_datastore']

In [2]:
# Read Data
churn_activities = pd.read_hdf(path_to_hdf_datastore, cfg['churn_activities_hdf_file'])

df = churn_activities
del churn_activities

df.head()

,Opportunity,Created By,Account ID,Company / Account,Contact,Lead,Priority,Activity Type,Task,Task/Event Record Type,...,Assigned,Date,Product Name,Assigned Role,Assigned Role Display,Created Date,Start,End,ECR Id,Parent ECR-ID
0,nan,Chinami Takebe,0010Y00001FvC5o,National Hospital Organization Hokkaido Medica...,???? Amano,nan,nan,Face to Face,0,Event,...,Chinami Takebe,2019-07-04T00:00,nan,HS-APAC-JP_N,nan,2019-07-04T00:00,2019-07-04T02:00,2019-07-04T02:30,ECR-593174,ECR-593174
1,SY_ELS_CK_2020_Mito Medical Center,Shiho Yakabe,0010Y00001FwK3c,Mito Medical Center,?? ??,nan,nan,nan,0,Event,...,Shiho Yakabe,2019-11-15T00:00,nan,HS-APAC-JP_C,nan,2019-12-04T00:00,2019-11-15T03:00,2019-11-15T04:00,ECR-202329,ECR-10122302
2,nan,David Lee,0010Y00001D1OCN,Yonsei University,???? ????,nan,nan,Virtual,0,Event,...,David Lee,2019-04-26T00:00,nan,A&G-APAC-KOREA-TAIWAN-CORE,nan,2019-04-29T00:00,2019-04-26T01:00,2019-04-26T02:00,ECR-137560,ECR-137560
3,DL_BF 19_Yonsei,David Lee,0010Y00001D1OCN,Yonsei University,???? ????,nan,nan,Face to Face,0,Event,...,David Lee,2019-04-17T00:00,nan,A&G-APAC-KOREA-TAIWAN-CORE,nan,2019-04-22T00:00,2019-04-17T06:00,2019-04-17T07:00,ECR-137560,ECR-137560
4,nan,Grace Kim,0013z00002LttHV,Daejeon Sun Hospital,???,nan,nan,Face to Face,0,Event,...,Grace Kim,2019-09-26T00:00,nan,RM-HS-APAC-KR,nan,2019-09-29T00:00,2019-09-26T02:30,2019-09-26T06:00,ECR-10325865,ECR-10325865


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown'], np.nan, inplace=True)
df.replace('.*\?.*', np.nan, regex=True, inplace=True)
df

,Opportunity,Created By,Account ID,Company / Account,Contact,Lead,Priority,Activity Type,Task,Task/Event Record Type,...,Assigned,Date,Product Name,Assigned Role,Assigned Role Display,Created Date,Start,End,ECR Id,Parent ECR-ID
0,NaN,Chinami Takebe,0010Y00001FvC5o,National Hospital Organization Hokkaido Medica...,NaN,NaN,NaN,Face to Face,0,Event,...,Chinami Takebe,2019-07-04T00:00,NaN,HS-APAC-JP_N,NaN,2019-07-04T00:00,2019-07-04T02:00,2019-07-04T02:30,ECR-593174,ECR-593174
1,SY_ELS_CK_2020_Mito Medical Center,Shiho Yakabe,0010Y00001FwK3c,Mito Medical Center,NaN,NaN,NaN,NaN,0,Event,...,Shiho Yakabe,2019-11-15T00:00,NaN,HS-APAC-JP_C,NaN,2019-12-04T00:00,2019-11-15T03:00,2019-11-15T04:00,ECR-202329,ECR-10122302
2,NaN,David Lee,0010Y00001D1OCN,Yonsei University,NaN,NaN,NaN,Virtual,0,Event,...,David Lee,2019-04-26T00:00,NaN,A&G-APAC-KOREA-TAIWAN-CORE,NaN,2019-04-29T00:00,2019-04-26T01:00,2019-04-26T02:00,ECR-137560,ECR-137560
3,DL_BF 19_Yonsei,David Lee,0010Y00001D1OCN,Yonsei University,NaN,NaN,NaN,Face to Face,0,Event,...,David Lee,2019-04-17T00:00,NaN,A&G-APAC-KOREA-TAIWAN-CORE,NaN,2019-04-22T00:00,2019-04-17T06:00,2019-04-17T07:00,ECR-137560,ECR-137560
4,NaN,Grace Kim,0013z00002LttHV,Daejeon Sun Hospital,NaN,NaN,NaN,Face to Face,0,Event,...,Grace Kim,2019-09-26T00:00,NaN,RM-HS-APAC-KR,NaN,2019-09-29T00:00,2019-09-26T02:30,2019-09-26T06:00,ECR-10325865,ECR-10325865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317789,NaN,[Inactive] Landon Bell,NaN,NaN,NaN,NaN,3-Medium,NaN,1,Task,...,[Inactive] Landon Bell,2019-02-08T00:00,NaN,Inactive,NaN,2019-02-08T00:00,2019-02-08T00:00,2019-02-08T00:00,NaN,NaN
317790,NaN,[Inactive] Landon Bell,NaN,NaN,NaN,NaN,3-Medium,NaN,1,Task,...,[Inactive] Landon Bell,2019-01-08T00:00,NaN,Inactive,NaN,2019-01-08T00:00,2019-01-08T00:00,2019-01-08T00:00,NaN,NaN
317791,NaN,[Inactive] Landon Bell,NaN,NaN,NaN,NaN,3-Medium,NaN,1,Task,...,[Inactive] Landon Bell,2019-01-10T00:00,NaN,Inactive,NaN,2019-01-10T00:00,2019-01-10T00:00,2019-01-10T00:00,NaN,NaN
317792,NaN,[Inactive] Landon Bell,NaN,NaN,NaN,NaN,3-Medium,NaN,1,Task,...,[Inactive] Landon Bell,2019-01-29T00:00,NaN,Inactive,NaN,2019-01-29T00:00,2019-01-29T00:00,2019-01-29T00:00,NaN,NaN


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

DC                                                              814
DS Digital Commons - Expert Gallery Suite                       422
DC - EGS                                                        313
2019 | Scopus Custom Data (REQUIRES NEW CONTRACT) | NSF | JM    271
eBooks 2019                                                     214
                                                               ... 
ELS_NS_2019_Yoshikawa Chuo Sogo Hospital                          1
Bartell Drugs - CDC - 2019 - CPPBCK Renew                         1
LHC Hospice-2019                                                  1
2019 MUSC new BS Subs                                             1
Intel_Knovel 2019 Renewal_Invoice ONLY                            1
Name: Opportunity, Length: 34295, dtype: int64
Jessemay Diah Tuyay         6854
Sadmin Sadmin               6771
B2BMA Integration           6732
Andrew Heldt                6671
Jamie Bridewell             6373
                            ... 
[Inacti

done                                                                                                                                                                                                                         2
eBook sales done and closed                                                                                                                                                                                                  2
Organize, with Miguel, a visit at this customer, to evaluate UTD national deal impact and possibility of CK subscription                                                                                                     2
Call                                                                                                                                                                                                                         2
if November also is flat lined they will go for the SDAC and cancel SDCE.                                   

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
23,Product Name,"[[nan, Digital Commons - Recurring, Knovel Tec...",5,object,100.00
18,Follow Up Notes,"[[nan, No answer in July, try in September, Ma...",179,object,99.94
17,Follow Up Subject,"[[nan, Chase for feedback, Check project statu...",473,object,99.84
19,Name of Value Prop,"[[nan, DVCRI Value Story, Clinical practice, C...",478,object,99.76
25,Assigned Role Display,"[[nan, AM - CS Americas 17, Credit Controller,...",9,object,99.61
13,Call Result,"[[nan, 260, called the main library number; Ja...",1698,object,99.46
14,Topics Discussed,"[[nan, Product feedback and features; Institut...",122,object,95.40
5,Lead,"[[nan, Gerald Crown, Christine Elliott, Kelley...",22488,object,86.08
11,Event Subtype,"[[Event, nan]]",2,object,66.65
0,Opportunity,"[[nan, SY_ELS_CK_2020_Mito Medical Center, DL_...",34296,object,58.08


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In this case, we will also manually drop Event Subtype, Opportunity, Contact, Full Comments, Comments and Company / Account.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove other columns
df = df.drop(columns=['Event Subtype', 'Opportunity', 'Contact', 'Full Comments', 'Comments', 'Company / Account'])

-------------------- Dropping Columns with missing data --------------------
Following 8 columns have missing data over the threshold and will be removed
Index(['Assigned Role Display', 'Call Result', 'Follow Up Notes',
       'Follow Up Subject', 'Lead', 'Name of Value Prop', 'Product Name',
       'Topics Discussed'],
      dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
3,Activity Type,"[[Face to Face, nan, Virtual, Online, Phone, E...",7,object,57.32
2,Priority,"[[nan, 1-ASAP, 3-Medium, 2-High, Low, 4-Low]]",6,object,33.35
6,Task Subtype,"[[nan, Task, Email, Call]]",4,object,33.35
15,ECR Id,"[[ECR-593174, ECR-202329, ECR-137560, ECR-1032...",25903,object,20.11
16,Parent ECR-ID,"[[ECR-593174, ECR-10122302, ECR-137560, ECR-10...",23167,object,20.11
1,Account ID,"[[0010Y00001FvC5o, 0010Y00001FwK3c, 0010Y00001...",25948,object,20.02
7,Subject,"[[Meeting, Email - EMBASE Usage, Meeting - BF ...",97922,object,3.81
11,Assigned Role,"[[HS-APAC-JP_N, HS-APAC-JP_C, A&G-APAC-KOREA-T...",317,object,2.16
13,Start,"[[2019-07-04T02:00, 2019-11-15T03:00, 2019-04-...",18379,object,0.63
14,End,"[[2019-07-04T02:30, 2019-11-15T04:00, 2019-04-...",20295,object,0.63


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

Activity Type - Impute with most frequent

Priority - Impute with most frequent

Task Subtype - Impute with most frequent

In [8]:
#Filling all missing values with 0
# data = data.fillna(0)

#Filling missing values with the most frequest values
df['Activity Type'].fillna(
    df['Activity Type'].value_counts().idxmax(), inplace=True)

df['Priority'].fillna(
    df['Priority'].value_counts().idxmax(), inplace=True)

df['Task Subtype'].fillna(
    df['Task Subtype'].value_counts().idxmax(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['Task'])

# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
4,Task,"[[0, 1]]",2,int64,0.00
5,Task/Event Record Type,"[[Event, Task, Mitigation]]",3,object,0.00
6,Task Subtype,"[[Email, Task, Call]]",3,object,0.00
2,Priority,"[[3-Medium, 1-ASAP, 2-High, Low, 4-Low]]",5,object,0.00
3,Activity Type,"[[Face to Face, Virtual, Online, Phone, Email,...",6,object,0.00


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
4,Task,"[[0, 1]]",2,int64,0.00
5,Task/Event Record Type,"[[Event, Task, Mitigation]]",3,object,0.00
6,Task Subtype,"[[Email, Task, Call]]",3,object,0.00
2,Priority,"[[3-Medium, 1-ASAP, 2-High, Low, 4-Low]]",5,object,0.00
3,Activity Type,"[[Face to Face, Virtual, Online, Phone, Email,...",6,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 1 features - **********************
 Task standardized to Task


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
print(columns_to_bin)

['Task', 'Task/Event Record Type', 'Task Subtype', 'Priority', 'Activity Type']


In [18]:
# Specify Columns to encode
columns_to_exclude = []
columns_to_include = columns_to_bin
columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
columns_to_encode

['Task', 'Task/Event Record Type', 'Task Subtype', 'Priority', 'Activity Type']

In [19]:
for column in columns_to_encode:
    encoded_columns = pd.get_dummies(df[column])
    print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
    df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

Encoding collumns : Task to 2 new encoded columns
Encoding collumns : Task/Event Record Type to 3 new encoded columns
Encoding collumns : Task Subtype to 3 new encoded columns
Encoding collumns : Priority to 5 new encoded columns
Encoding collumns : Activity Type to 6 new encoded columns


In [20]:
df.shape

(317794, 31)

In [21]:
df.head()

,Created By,Account ID,Subject,Activity ID,Assigned,Date,Assigned Role,Created Date,Start,End,...,2-High,3-Medium,4-Low,Low,Email_Activity Type,Face to Face,Online,Other,Phone,Virtual
0,Chinami Takebe,0010Y00001FvC5o,Meeting,00U1v00000SNwK8,Chinami Takebe,2019-07-04T00:00,HS-APAC-JP_N,2019-07-04T00:00,2019-07-04T02:00,2019-07-04T02:30,...,0,1,0,0,0,1,0,0,0,0
1,Shiho Yakabe,0010Y00001FwK3c,Meeting,00U3z00000rZouU,Shiho Yakabe,2019-11-15T00:00,HS-APAC-JP_C,2019-12-04T00:00,2019-11-15T03:00,2019-11-15T04:00,...,0,1,0,0,0,0,0,0,0,1
2,David Lee,0010Y00001D1OCN,Email - EMBASE Usage,00U1v00000QZ6eu,David Lee,2019-04-26T00:00,A&G-APAC-KOREA-TAIWAN-CORE,2019-04-29T00:00,2019-04-26T01:00,2019-04-26T02:00,...,0,1,0,0,0,0,0,0,0,1
3,David Lee,0010Y00001D1OCN,Meeting - BF Proposal,00U1v00000QG3jm,David Lee,2019-04-17T00:00,A&G-APAC-KOREA-TAIWAN-CORE,2019-04-22T00:00,2019-04-17T06:00,2019-04-17T07:00,...,0,1,0,0,0,1,0,0,0,0
4,Grace Kim,0013z00002LttHV,Meeting,00U3z00000qky6Z,Grace Kim,2019-09-26T00:00,RM-HS-APAC-KR,2019-09-29T00:00,2019-09-26T02:30,2019-09-26T06:00,...,0,1,0,0,0,1,0,0,0,0


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = ['Start', 'End', 'Date', 'Created Date']

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calculate new variable activity_length
df['activity_length'] = months_between_columns(df['End'],df['Start'])

In [25]:
 df.head()

,Created By,Account ID,Subject,Activity ID,Assigned,Date,Assigned Role,Created Date,Start,End,...,3-Medium,4-Low,Low,Email_Activity Type,Face to Face,Online,Other,Phone,Virtual,activity_length
0,Chinami Takebe,0010Y00001FvC5o,Meeting,00U1v00000SNwK8,Chinami Takebe,2019-07-04,HS-APAC-JP_N,2019-07-04,2019-07-04 02:00:00,2019-07-04 02:30:00,...,1,0,0,0,1,0,0,0,0,0.00
1,Shiho Yakabe,0010Y00001FwK3c,Meeting,00U3z00000rZouU,Shiho Yakabe,2019-11-15,HS-APAC-JP_C,2019-12-04,2019-11-15 03:00:00,2019-11-15 04:00:00,...,1,0,0,0,0,0,0,0,1,0.00
2,David Lee,0010Y00001D1OCN,Email - EMBASE Usage,00U1v00000QZ6eu,David Lee,2019-04-26,A&G-APAC-KOREA-TAIWAN-CORE,2019-04-29,2019-04-26 01:00:00,2019-04-26 02:00:00,...,1,0,0,0,0,0,0,0,1,0.00
3,David Lee,0010Y00001D1OCN,Meeting - BF Proposal,00U1v00000QG3jm,David Lee,2019-04-17,A&G-APAC-KOREA-TAIWAN-CORE,2019-04-22,2019-04-17 06:00:00,2019-04-17 07:00:00,...,1,0,0,0,1,0,0,0,0,0.00
4,Grace Kim,0013z00002LttHV,Meeting,00U3z00000qky6Z,Grace Kim,2019-09-26,RM-HS-APAC-KR,2019-09-29,2019-09-26 02:30:00,2019-09-26 06:00:00,...,1,0,0,0,1,0,0,0,0,0.00


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = ['Subject','Assigned Role']

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,Created By,Account ID,Activity ID,Assigned,Date,Created Date,Start,End,ECR Id,Parent ECR-ID,...,3-Medium,4-Low,Low,Email_Activity Type,Face to Face,Online,Other,Phone,Virtual,activity_length
0,Chinami Takebe,0010Y00001FvC5o,00U1v00000SNwK8,Chinami Takebe,2019-07-04,2019-07-04,2019-07-04 02:00:00,2019-07-04 02:30:00,ECR-593174,ECR-593174,...,1,0,0,0,1,0,0,0,0,0.00
1,Shiho Yakabe,0010Y00001FwK3c,00U3z00000rZouU,Shiho Yakabe,2019-11-15,2019-12-04,2019-11-15 03:00:00,2019-11-15 04:00:00,ECR-202329,ECR-10122302,...,1,0,0,0,0,0,0,0,1,0.00
2,David Lee,0010Y00001D1OCN,00U1v00000QZ6eu,David Lee,2019-04-26,2019-04-29,2019-04-26 01:00:00,2019-04-26 02:00:00,ECR-137560,ECR-137560,...,1,0,0,0,0,0,0,0,1,0.00
3,David Lee,0010Y00001D1OCN,00U1v00000QG3jm,David Lee,2019-04-17,2019-04-22,2019-04-17 06:00:00,2019-04-17 07:00:00,ECR-137560,ECR-137560,...,1,0,0,0,1,0,0,0,0,0.00
4,Grace Kim,0013z00002LttHV,00U3z00000qky6Z,Grace Kim,2019-09-26,2019-09-29,2019-09-26 02:30:00,2019-09-26 06:00:00,ECR-10325865,ECR-10325865,...,1,0,0,0,1,0,0,0,0,0.00


Save clean data to hdf store

In [29]:
store = HDFStore(path_to_clean_hdf_datastore)
df.loc[:, ['Created By', 'Account ID', 'Activity ID', 'Assigned', 'ECR Id', 'Parent ECR-ID']] = df.loc[:, ['Created By', 'Account ID', 'Activity ID', 'Assigned', 'ECR Id', 'Parent ECR-ID']].astype(str)
store.put(key=cfg['churn_activities_hdf_file']+"_clean", value=df)
store.close()


C:\Users\LEIGHA\.virtualenvs\rss-customer-churn-JupVR20Y\lib\site-packages\IPython\core\interactiveshell.py:3319: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\LEIGHA\.virtualenvs\rss-customer-churn-JupVR20Y\lib\site-packages\IPython\core\interactiveshell.py:3319: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_items] [items->None]

  exec(code_obj, self.user_global_ns, self.user_ns)
